In [2]:
import sys
sys.path.append('src')
from sqlalchemy import create_engine


In [3]:
def mysql_engine(user = 'root', password = 'abc', host = '127.0.0.1', port = '3306', database = 'a001_db'):
    engine = create_engine("mysql://{0}:{1}@{2}:{3}/{4}?charset=utf8".format(user, password, host, port, database))
    return engine
engine = mysql_engine(user='mogilevd', password='Rivka2015!', host='5.2.16.131', database='football_data')


In [5]:
import pandas as pd

#df=pd.read_sql_query("SELECT * FROM whoscored_data WHERE team='Chelsea' and MATCH_DATE IN (SELECT MAX(MATCH_DATE) FROM whoscored_data WHERE TEAM='Chelsea')", engine)
df=pd.read_sql_query("SELECT * FROM fbref2 WHERE squad='chelsea' and season=2021 and comp='Premier League'", engine)

In [53]:
df['xt_idx_x']=pd.cut(df['x'], 12, labels=range(12))
df['xt_idx_y']=pd.cut(df['y'], 8, labels=range(8))
df['xT']=df.apply(lambda row: xt_grid[row['xt_idx_y']][row['xt_idx_x']] if row['xt_idx_x'] in range(12) and row['xt_idx_y'] in range(8) else 0, axis=1)

df['end_xt_idx_x']=pd.cut(df['endX'], 12, labels=range(12))
df['end_xt_idx_y']=pd.cut(df['endY'], 8, labels=range(8))
df['end_xT']=df.apply(lambda row: xt_grid[row['end_xt_idx_y']][row['end_xt_idx_x']] if row['end_xt_idx_x'] in range(12) and row['end_xt_idx_y'] in range(8) else 0, axis=1)
df['xT_pass']=df['end_xT']-df['xT']

In [64]:
df.columns

Index(['id', 'eventId', 'minute', 'second', 'teamId', 'x', 'y',
       'expandedMinute', 'qualifiers', 'isTouch', 'period', 'event_type',
       'outcomeType', 'playerId', 'endX', 'endY', 'relatedEventId',
       'relatedPlayerId', 'goalMouthZ', 'goalMouthY', 'isShot', 'blockedX',
       'blockedY', 'isGoal', 'cardType', 'matchId', 'player_name',
       'match_date', 'season', 'related_player_name', 'match_seconds',
       'satisfied_events', 'team', 'opponent', 'is_home_team', 'shirt_number',
       'formation', 'position', 'pass_receiver', 'pass_receiver_position',
       'pass_receiver_shirt_number', 'isOwnGoal', 'league', 'xt_idx_x',
       'xt_idx_y', 'xT', 'end_xt_idx_x', 'end_xt_idx_y', 'end_xT', 'xT_diff',
       'xT_pass'],
      dtype='object')

In [51]:
df.loc[(df['event_type']==1)&(df['player_name']=='romelu lukaku')][['x','y','endX','endY','xT','end_xT','xT_diff', 'outcomeType']]

,x,y,endX,endY,xT,end_xT,xT_diff,outcomeType
991,79.8,56.3,80.3,70.2,0.035113,0.028552,-0.006561,1
1092,69.2,57.2,67.1,62.4,0.023851,0.023851,0.000000,1
1098,60.5,32.2,54.4,33.6,0.019351,0.016856,-0.002495,1
1123,63.4,66.2,56.6,70.7,0.019351,0.016856,-0.002495,1
1226,70.7,54.5,69.5,72.4,0.023851,0.024122,0.000271,1
...,...,...,...,...,...,...,...,...
25991,58.9,56.9,46.8,62.4,0.019971,0.014846,-0.005125,1
26007,51.2,76.0,49.0,67.5,0.016118,0.014291,-0.001827,1
26104,78.4,72.1,73.4,84.1,0.028552,0.024015,-0.004537,1
26118,57.6,25.2,58.7,33.8,0.016856,0.019351,0.002495,0


In [63]:
df.loc[df['event_type']==1].groupby('player_name').agg({'xT_pass':'sum'}).sort_values(by='xT_pass', ascending=False)


,xT_pass
player_name,
mason mount,18.889689
marcos alonso,13.475573
hakim ziyech,13.182317
reece james,12.056981
cesar azpilicueta,9.373128
antonio rudiger,8.210000
ben chilwell,5.138050
callum hudson odoi,4.757814
thiago silva,4.445345


In [72]:
df_pass_received = df.loc[(df['event_type']==1)&(df['outcomeType']==1)].groupby('pass_receiver').agg({'xT_pass':'sum'}).sort_values('xT_pass', ascending=False).rename(columns={'xT_pass':'xT_pass_completed'}).reset_index()
df_pass_received.loc[df_pass_received['pass_receiver'].isin(df['player_name'].unique())]

,pass_receiver,xT_pass_completed
0,kai havertz,5.867907
1,romelu lukaku,4.048077
2,mason mount,3.726290
3,christian pulisic,3.077657
4,timo werner,3.042159
5,marcos alonso,2.136420
6,hakim ziyech,2.037149
7,reece james,1.789627
8,ruben loftus cheek,1.753680
9,cesar azpilicueta,1.550385


In [6]:
from footmav import FbRefData, aggregate_by, fb, per_90
from footmav.data_definitions.derived import function_attribute
from footmav.data_definitions import attribute_functions as F

In [7]:
data = FbRefData(df)

In [8]:
@function_attribute
def NON_DEF_TOUCHES():
  return F.Col(fb.TOUCHES_MID_3RD)+F.Col(fb.TOUCHES_ATT_3RD)


In [9]:
data.pipe(aggregate_by, [fb.PLAYER]).with_attributes(NON_DEF_TOUCHES).df[[fb.PLAYER.N, NON_DEF_TOUCHES.N]]

,player,non_def_touches
0,andreas christensen,782
1,antonio rudiger,1511
2,ben chilwell,346
3,callum hudson odoi,509
4,cesar azpilicueta,1168
5,christian pulisic,508
6,edouard mendy,4
7,emerson palmieri,2
8,hakim ziyech,756
9,jorginho,1422


In [18]:
np.linspace(0, 100, 13)

array([  0.        ,   8.33333333,  16.66666667,  25.        ,
        33.33333333,  41.66666667,  50.        ,  58.33333333,
        66.66666667,  75.        ,  83.33333333,  91.66666667,
       100.        ])

In [47]:
import pandas as pd

x0=pd.cut([10, 25, 15, 20], bins=np.linspace(0, 100, 13),labels=range(0,12)).to_list()
print(x0)

[1, 2, 1, 2]


In [48]:
x1=pd.cut([10, 29, 20, 12], bins=np.linspace(0, 100, 13),labels=range(0,12)).to_list()
print(x1)

[1, 3, 2, 1]


In [49]:
y0=pd.cut([10, 2, 20, 12], bins=np.linspace(0, 100, 9),labels=range(0,8)).to_list()
print(y0)

[0, 0, 1, 0]


In [52]:
y1=pd.cut( [87, 2, 36, 4], bins=np.linspace(0, 100, 9),labels=range(0,8)).to_list()
print(y1)

[6, 0, 2, 0]


In [51]:
[grid[y1[i]][x1[i]]-grid[y0[i]][x0[i]] for i in range(4)]

[24, 1, 5, -1]

In [28]:
grid = [
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
        [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
        [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
        [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28],
        [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32],
        [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36],
        [29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40],
    ]

np.linspace(0, 100, 9)

array([  0. ,  12.5,  25. ,  37.5,  50. ,  62.5,  75. ,  87.5, 100. ])